## The “Budget Keeper” (Token Economics)

In [1]:
import pandas as pd
import sys
sys.path.append('..')
from  utils.token_utils import count_messages_tokens

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model
from IPython.display import Markdown, display

In [2]:
#Load messages from file
file_path = "../data/sample_messages.txt"

with open(file_path, "r", encoding="utf-8") as f:
    messages = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(messages)} messages")
messages[:3]

Loaded 50 messages


['BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.',
 'SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.',
 'Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.']

In [3]:
model = pick_model('openai', 'general')
client = LLMClient('openai', model)


def overflow_summarize(message, max_tokens_context=60):
    
    prompt_text, spec = render(
    'overflow_summarize.v1',
    max_tokens_context ='max_tokens_context',
    context=message,
    task='Produce a concise, safe summary suitable for disaster response processing.',
    format='SUMMARY: <text>'
    )

    messages = [{'role': 'user', 'content': prompt_text}]
    response = client.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

    return response["text"].strip()
